In [1]:
import numpy as np
import pandas as pd
import os

from rdkit import Chem
from rdkit.Chem import MACCSkeys, rdFingerprintGenerator
from rdkit import DataStructs

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import optuna
import joblib

In [2]:
np.random.seed(1234)

In [3]:
endpoint = 'skin-sensitization'
# endpoint = 'eye-irritation'

loc = r'D:\School\Semester3\Seminar - Reproducibility\seminar-toxicity\data'
endpoint_loc = os.path.join(loc, endpoint)
model = r'D:\School\Semester3\Seminar - Reproducibility\seminar-toxicity\src\models'
model_loc = os.path.join(model, endpoint)

In [4]:
filename = 'train.csv'
df_train = pd.read_csv(os.path.join(endpoint_loc, filename))

In [5]:
df_train.shape

(2956, 2)

In [6]:
df_train.head()

,SMILES,Activity
0,CC(=O)CC(=O)NC1=CC(Cl)=CC=C1OC,0
1,CC1=NN2C(=NC3C=CC=CC=3C2=O)C1N=NC1C=CC(Cl)=CC=...,0
2,CCCCCCCCCCCCCCCCCCN(CCCCCCCCCCCCCCCCCC)C(=O)C1...,0
3,ClC1C=C(C=CC=1)OC1C=CC(Cl)=CC=1,0
4,OCCN1CCN(CCS(O)(=O)=O)CC1,0


In [7]:
filename = 'val.csv'
df_val = pd.read_csv(os.path.join(endpoint_loc, filename))

In [8]:
df_val.shape

(739, 2)

In [9]:
df_val.head()

,SMILES,Activity
0,CCC1(CO)COCOC1,0
1,CC(=O)C(=CC1C=CC=CC=1[N+]([O-])=O)C(=O)OC,1
2,[O-][N+](=O)C1C(=NN2C=CC=CC2=1)OCCO,0
3,CCCCCCCCCCCC(=O)OC1CC(C)(C)NC(C)(C)C1,0
4,CC(C)(C)CC(C)CC(=O)OOC(C)(C)C,1


In [10]:
train_smiles = df_train['SMILES'].to_numpy()
train_labels = df_train['Activity'].to_numpy()
val_smiles = df_val['SMILES'].to_numpy()
val_labels = df_val['Activity'].to_numpy()

In [11]:
print('train size smiles :', train_smiles.shape)
print('train size labels :', train_labels.shape)
print('pos samples in train size :', train_labels[train_labels == 1].shape)
print('neg samples in train size :', train_labels[train_labels == 0].shape)
print('val size smiles :', val_smiles.shape)
print('val size labels :', val_labels.shape)
print('pos samples in val size :', val_labels[val_labels == 1].shape)
print('neg samples in val size :', val_labels[val_labels == 0].shape)

train size smiles : (2956,)
train size labels : (2956,)
pos samples in train size : (1617,)
neg samples in train size : (1339,)
val size smiles : (739,)
val size labels : (739,)
pos samples in val size : (404,)
neg samples in val size : (335,)


In [12]:
def get_MAACS(smiles_array, labels):
    fps = []
    y = []
    for smiles, label in zip(smiles_array, labels):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            pass
        else:
            fps.append(np.array(MACCSkeys.GenMACCSKeys(mol)))
            y.append(label)

    assert len(fps) == len(y)
    
    return np.array(fps), np.array(y)

In [13]:
train_fingerprints, train_labels = get_MAACS(train_smiles, train_labels)
val_fingerprints, val_labels = get_MAACS(val_smiles, val_labels)

[13:25:16] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[13:25:17] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 9 10 11 13 14 15


In [14]:
print('train size fingerprints :', train_fingerprints.shape)
print('train size labels :', train_labels.shape)
print('pos samples in train size :', train_labels[train_labels == 1].shape)
print('neg samples in train size :', train_labels[train_labels == 0].shape)
print('val size fingerprints :', val_fingerprints.shape)
print('val size labels :', val_labels.shape)
print('pos samples in val size :', val_labels[val_labels == 1].shape)
print('neg samples in val size :', val_labels[val_labels == 0].shape)

train size fingerprints : (2956, 167)
train size labels : (2956,)
pos samples in train size : (1617,)
neg samples in train size : (1339,)
val size fingerprints : (737, 167)
val size labels : (737,)
pos samples in val size : (403,)
neg samples in val size : (334,)


In [15]:
def objective(trial, xtrain, ytrain):
    n = trial.suggest_int('n_estimators', 50, 250)
    rf = RandomForestClassifier(n_estimators = n)

    scores = cross_validate(rf, xtrain, ytrain, cv=5, scoring='roc_auc')
    mean_roc = scores['test_score'].mean()

    return 1/(mean_roc + 1e-6)

In [17]:
study = optuna.create_study(study_name='rf_study_skin', storage='sqlite:///rf_study_skin.db')  # Create a new study.
study.optimize(lambda trial: objective(trial, train_fingerprints, train_labels), n_trials=20)  # Invoke optimization of the objective function.

[I 2024-01-14 13:25:50,950] A new study created in RDB with name: rf_study_skin
[I 2024-01-14 13:25:54,529] Trial 0 finished with value: 1.3291156459135878 and parameters: {'n_estimators': 167}. Best is trial 0 with value: 1.3291156459135878.
[I 2024-01-14 13:25:56,074] Trial 1 finished with value: 1.332445743318067 and parameters: {'n_estimators': 68}. Best is trial 0 with value: 1.3291156459135878.
[I 2024-01-14 13:25:58,728] Trial 2 finished with value: 1.3288886621694984 and parameters: {'n_estimators': 122}. Best is trial 2 with value: 1.3288886621694984.
[I 2024-01-14 13:26:00,030] Trial 3 finished with value: 1.3424355147477416 and parameters: {'n_estimators': 55}. Best is trial 2 with value: 1.3288886621694984.
[I 2024-01-14 13:26:02,546] Trial 4 finished with value: 1.3261811037837352 and parameters: {'n_estimators': 104}. Best is trial 4 with value: 1.3261811037837352.
[I 2024-01-14 13:26:04,911] Trial 5 finished with value: 1.3354666198959269 and parameters: {'n_estimators':

In [18]:
study.best_params

{'n_estimators': 104}

In [19]:
rf = RandomForestClassifier(n_estimators = 104, random_state=1234)
rf.fit(train_fingerprints, train_labels)

RandomForestClassifier(n_estimators=104, random_state=1234)

In [20]:
# performing predictions on the test dataset 
y_pred = rf.predict(train_fingerprints)

In [21]:
print('Train accuracy = ', (y_pred == train_labels).sum()/len(train_labels))

Train accuracy =  0.9905277401894452


In [22]:
y_pred = rf.predict(val_fingerprints)

In [23]:
print('Val accuracy = ', (y_pred == val_labels).sum()/len(val_labels))

Val accuracy =  0.6648575305291723


In [24]:
tn, fp, fn, tp = confusion_matrix(y_pred, val_labels).ravel()

In [25]:
(tn, fp, fn, tp)

(197, 110, 137, 293)

In [26]:
ACC = (tp + tn)/(tp + tn + fn + fp)
SEN = tp/(tp + fn)
SPE = tn/(tn + fp)

In [27]:
print(f'Accuracy = {ACC}')
print(f'Sensitivity = {SEN}')
print(f'Specificity = {SPE}')

Accuracy = 0.6648575305291723
Sensitivity = 0.6813953488372093
Specificity = 0.6416938110749185


#### Saving model

In [28]:
model_name = 'rf-MAACS.joblib'
joblib.dump(rf, os.path.join(model_loc, model_name))

['D:\\School\\Semester3\\Seminar - Reproducibility\\seminar-toxicity\\src\\models\\skin-sensitization\\rf-MAACS.joblib']